## Hi This is my data preparation worksheet

In [30]:
with open("the-verdict.txt", "r", encoding="utf-8") as f:
      raw_text = f.read()

print("Total number of characters: ", len(raw_text))
raw_text[:99]

Total number of characters:  20479


'I HAD always thought Jack Gisburn rather a cheap genius--though a good fellow enough--so it was no '

In [31]:
import re

In [32]:
text = "Hello I am just trying this out!"
result = re.split(r'(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [33]:
result = re.split(r'[,.]|(\s)', text)
print(result)

['Hello', ' ', 'I', ' ', 'am', ' ', 'just', ' ', 'trying', ' ', 'this', ' ', 'out!']


In [34]:
result = [item for item in result if item.strip()]
result

['Hello', 'I', 'am', 'just', 'trying', 'this', 'out!']

This is a simple example of how a tokenizer works

In [35]:
preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',raw_text)

In [36]:
preprocessed = [item for item in preprocessed if item.strip()]
print(preprocessed[:30])

['I', 'HAD', 'always', 'thought', 'Jack', 'Gisburn', 'rather', 'a', 'cheap', 'genius', '--', 'though', 'a', 'good', 'fellow', 'enough', '--', 'so', 'it', 'was', 'no', 'great', 'surprise', 'to', 'me', 'to', 'hear', 'that', ',', 'in']


In [37]:
print(len(preprocessed))

4690


Convert Tokens into Token IDs

In [38]:
allwords = sorted(set(preprocessed))

In [39]:
vocab_size = len(allwords)
vocab_size

1130

In [40]:
vocab = {token:integer for integer,token in enumerate(allwords)}

In [41]:
for i,item in enumerate(vocab.items()):
    print(i,"->",item)
    if i>=20:
        break

0 -> ('!', 0)
1 -> ('"', 1)
2 -> ("'", 2)
3 -> ('(', 3)
4 -> (')', 4)
5 -> (',', 5)
6 -> ('--', 6)
7 -> ('.', 7)
8 -> (':', 8)
9 -> (';', 9)
10 -> ('?', 10)
11 -> ('A', 11)
12 -> ('Ah', 12)
13 -> ('Among', 13)
14 -> ('And', 14)
15 -> ('Are', 15)
16 -> ('Arrt', 16)
17 -> ('As', 17)
18 -> ('At', 18)
19 -> ('Be', 19)
20 -> ('Begin', 20)


In [42]:
class SimpleTokenizerV1:
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item for item in preprocessed if item.strip()]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1',text)
        return text

In [43]:
tokenizer = SimpleTokenizerV1(vocab)

text = """"It's the last he painted, you know,"
           Mrs. Gisburn said with pardonable pride."""
ids = tokenizer.encode(text)
print(ids)

[1, 56, 2, 850, 988, 602, 533, 746, 5, 1126, 596, 5, 1, 67, 7, 38, 851, 1108, 754, 793, 7]


In [44]:
tokenizer.decode(ids)

'" It\' s the last he painted, you know," Mrs. Gisburn said with pardonable pride.'

In [45]:
sampleText = "Hello, do you like tea?"
print(tokenizer.encode(sampleText))

KeyError: 'Hello'

Adding Special Context tokens

In [46]:
alltokens = sorted(set(preprocessed))
alltokens.extend(["<|endoftext|>", "<|unk|>"])
vocab = {token:integer for integer,token in enumerate(alltokens)}

In [47]:
len(vocab.items())

1132

In [50]:
class SimpleTokenizerV2 :
    def __init__(self, vocab):
        self.str_to_int = vocab
        self.int_to_str = {i:s for s,i in vocab.items()}

    def encode(self,text):
        preprocessed = re.split(r'([,.:;?_!"()\']|--|\s)',text)
        preprocessed = [item.strip() for item in preprocessed if item.strip()]
        preprocessed [
            item if item in self.str_to_int
            else [ "<|unk|>" for item in preprocessed ]
        ]
        ids = [self.str_to_int[s] for s in preprocessed]
        return ids

    def decode(self, ids):
        text = " ".join([self.int_to_str[i] for i in ids])
        text = re.sub(r'\s+([,.:;?_!"()\'])', r'\1',text)
        return text

In [52]:
tokenizer = SimpleTokenizerV2
text1 = "Hello, do you like tea?"
text2 = "In the sunlit terraces of the palace."
text = " <|endoftext|> ".join((text1,text2))

print(text)

Hello, do you like tea? <|endoftext|> In the sunlit terraces of the palace.


In [53]:
tokenizer.encode(text)

TypeError: SimpleTokenizerV2.encode() missing 1 required positional argument: 'text'

## Byte Pair Encoding (BPE)

Used in GPT-2 and GPT-3

In [54]:
pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/894.9 kB ? eta -:--:--
   ---------------------------------------- 894.9/894.9 kB 5.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [55]:
import importlib
import tiktoken

print("tiktoken version:", importlib.metadata.version("tiktoken"))

tiktoken version: 0.9.0


In [56]:
tokenizer = tiktoken.get_encoding("gpt2")

In [57]:
text = ("Hello, do you like tea? <|endoftext|> In the sunlit terraces"
        "of someunknownPlace.")

In [58]:
integers = tokenizer.encode(text, allowed_special = {"<|endoftext|>"})

print(integers)

[15496, 11, 466, 345, 588, 8887, 30, 220, 50256, 554, 262, 4252, 18250, 8812, 2114, 1659, 617, 34680, 27271, 13]


In [59]:
 strings = tokenizer.decode(integers)
strings

'Hello, do you like tea? <|endoftext|> In the sunlit terracesof someunknownPlace.'